# Stream Ciphers

In [1]:
import streamcipher
from streamcipher import LFSR
from streamcipher import berlekamp_massey
from streamcipher import ShrinkingGenerator
import numpy as np
from functools import reduce
from itertools import compress
from operator import xor

## Introduction

**Stream Ciphers** are symmetric key algorithms that encrypt the plaintext combining it with the **keystream**, a sequence of bits obtained from the key, typically applying a bitwise XOR operation. Due to the fact that bits can be seen as elements of GF(2), the inverse of the XOR operation is the XOR operation itself, then decryption is achieved by applying the same bitwise XOR operation to the ciphertext. 

To avoid to have a key that is as long as the plaintext/ciphertext, the keystream is typically produced using a **Pseudo Random Number Generators (PRNG)**, commonly based on **Linear Feedback Shift Registers (LFSR)**.

![Stream Cipher general structure](../Report%20Images/stream%20cipher.jpg)

In this notebook one will see the implementation of an LFSR together with the application of the Berlekamp-Massey algorithm to break it. Eventually a Shrinking Generator will be constructed.

## LFSR

A **Linear Feedback Shift Register (LFSR)** is a widely used component in cryptographic systems to generate pseudorandom binary sequences. It consists of a shift register whose input bit is determined by a linear combination of its previous state: this combination is performed using a feedback function, typically encoded as a polynomial, that selects certain state's bits to be XORed together and fed back into the register.

The output sequence is always periodic and its period exclusively depends on the structure of the LFSR. As different LFSRs can produce identical sequences, the optimal design prioritizes the shortest register length.

![Linear Feedback Shift Regsiter general structure](../Report%20Images/lfsr.jpg)

### LFSR implementation

Test LFSR implementation (all attributes and mehtods) with the polynomial `poly = [4, 1, 0]`.

In [15]:
lfsr = LFSR([4, 1, 0])
lfsr.__str__()
print(lfsr.cycle())
print(lfsr.run_steps(6))
lfsr.__str__()
for n, b in enumerate(lfsr):
    if n == 20:
        break
    print(b)


LFSR Initialization
poly: [1, 0, 0, 1]
init state: [1, 1, 1, 1]

Feedback polynomial: [0, 1, 4]
LFSR length: 4            
Current state: b'\x0f'

Cycle completed. 15 elements
[1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0]

Here are 6 iterations of the LFSR
[1, 1, 1, 1, 0, 1]

Feedback polynomial: [0, 1, 4]
LFSR length: 4            
Current state: b'\x06'
0
1
1
0
0
1
0
0
0
1
1
1
1
0
1
0
1
1
0
0


## Berlekamp Massey

### testing

Test the implementation Berlekamp-Massy Algorithm with the sequence produced by the following LFSR:
- `poly = [3, 1, 0]` and `state = '\x06'`
- `poly = [5, 2, 1, 0]` and `state = '\x05'`
- `poly = [96, 81, 17, 15, 0]` and `state = b'streamcipher'`

In [3]:
lfsr = LFSR([3, 1, 0], b'\x06')
bit_sequence = lfsr.cycle()
# print(bit_sequence)
berlekamp_massey(bit_sequence)

LFSR Initialization
poly: [1, 0, 1]
init state: [1, 1, 0]

Cycle completed. 7 elements


[0, 1, 3]

In [4]:
lfsr = LFSR([1, 5, 2, 0], b'\x05')
bit_sequence = lfsr.cycle()
# print(bit_sequence)
berlekamp_massey(bit_sequence)

LFSR Initialization
poly: [1, 1, 0, 0, 1]
init state: [0, 0, 1, 0, 1]

Cycle completed. 7 elements


[0, 1, 3]

In [5]:
lfsr = LFSR([0, 17, 81, 15, 95], b'streamcipher')
# The cycle takes too long to be computed. For time reasons the sequence is 
#made of the minimum amount of bits to correctly guess the feedback polynomial
bit_sequence = lfsr.run_steps(190)
# print(bit_sequence)
berlekamp_massey(bit_sequence)

LFSR Initialization
poly: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
init state: [1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0]

Here are 190 iterations of the LFSR


[0, 15, 17, 81, 95]

Load `'binary_sequence'` and find the polynomial corresponding to the shortest LFSR that can produce that sequence.

In [6]:
with open('binary_sequence.bin', 'rb') as f:
    bit_sequence = f.read()
bit_sequence = streamcipher.bytes_to_bool_list(bit_sequence)
# print(bit_sequence)
berlekamp_massey(bit_sequence)

[0, 2, 9, 12, 13, 14, 16]

## LFSR-based Generator (Shrinking Generator)

In [7]:
group_name = b'CipherCrafters'
group_name_bits = streamcipher.bytes_to_bool_list(group_name)
parity_bit = reduce(xor, group_name_bits)
parity_bit

0

### testing

Test Generator implementation (all attributes and methods).

In [8]:
sh_gen = ShrinkingGenerator(b'\x06', b'\x05')
g_seq = sh_gen.run_steps(20)
print(g_seq)

LFSR Initialization
poly: [0, 1, 0, 0, 1]
init state: [0, 0, 1, 1, 0]
LFSR Initialization
poly: [1, 0, 1]
init state: [1, 0, 1]

Here are 20 iterations of the Shrinking Generator
[0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0]


Decrypt the ciphertext in `ciphertext_shrinking.bin` or `ciphertext_selfshrinking.bin`.

In [9]:
polyA = [16, 15, 12, 10, 0]
polyS = [24, 11, 5, 2, 0]
stateA = b'\xc5\xd7' 
stateS = b'\x14\x84\xf8'

sh_gen = ShrinkingGenerator(stateA, stateS, polyA, polyS)

LFSR Initialization
poly: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]
init state: [1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1]
LFSR Initialization
poly: [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
init state: [0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0]


In [10]:
with open('ciphertext_shrinking.bin', 'rb') as f:
    ciphertext_shrinking = f.read()

In [11]:
ciphertext_shrinking = streamcipher.bytes_to_bool_list(ciphertext_shrinking)
sh_seq = sh_gen.run_steps(len(ciphertext_shrinking))

shrinking_plaintext = [cipher ^ gen for cipher, gen in zip(ciphertext_shrinking, sh_seq)]
shrinking_plaintext = streamcipher.bool_list_to_bytes(shrinking_plaintext)

print(shrinking_plaintext)


Here are 4992 iterations of the Shrinking Generator
b'The Shrinking Generator\nDon Coppersmith, Hugo Krawczyk, Yishay Mansour\nIBM T.J. Watson Research Center\nYorktown Heights, NY 10598\n\nAbstract. We present a new construction of a pseudorandom generator based on a simple combination of two LFSRs. The construction bas attractive properties as simplicity (conceptual and implementation-wise), scalability (hardware and security), proven minimal security conditions (exponential period, exponential linear complexity, good statistical properties), and resistance to known attacks. The construction is suitable for practical implementation of efficient stream cipher cryptosystems.'


## Conclusion